In [25]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from TimeSeriesDataset import TimeSeriesDataset
from utils import convert_to_year, download_data, weighted_mean, convert_to_year, is_same, compare
import pandas as pd
import numpy as np

dataset = TimeSeriesDataset(categories=['sc'], file_path="../datasets")
dataset.debug_clean_dataset()


polity_df = download_data("https://seshatdata.com/api/core/polities/?page_size=100")

Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories from 1800.0 to 1800.0
Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories_from from 1800.0 to 1800.0
Polity gb_british_emp_2 (1900.0 - 1900.0) is inactive but has data for sc/polity-territories_to from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700.0 - 1700.0) is inactive but has data for sc/polity-populations from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations_from from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700.0 - 1700.0) is inactive but has data for sc/polity-populations_from from 1800.0 to 1800.0
Polity jp_ashikaga (1400.0 - 1400.0) is inactive but has data for sc/polity-populations_to from 1500.0 to 1500.0
Polity us_early_illinois_confederation (1700

download old data and make it conform to TSD formatting

In [1]:
import pyreadr
data = pyreadr.read_r("/Users/mperuzzo/Documents/PT_analysis/fromJim2/sql_data_13may24_TableDat_5n_100yrs.Rdata")

In [27]:
raw_dat = data["datawide.mean"]
name_mapping = {}
for col in raw_dat.columns:
    new_name = col.replace('.', '-')
    name_mapping[col] = new_name.lower()

# rename the columns to make them match between datasets
raw_dat = raw_dat.rename(columns=name_mapping)
raw_dat.nga = raw_dat.nga.apply(lambda x: x[4:])
raw_dat = raw_dat.rename(columns={'nga': 'polityname', 
                                  'nga-time': 'year', 
                                  'polity-population':'polity-populations', 
                                  'polity-territory':'polity-territories', 
                                  'population-of-the-largest-settlement':'population-of-the-largest-settlements',
                                  'settlement-hierarchy':'settlement-hierarchies',
                                  'professional-priesthood':'professional-priesthoods',
                                  'examination-system':'examination-systems',
                                  'formal-legal-code':'formal-legal-codes',
                                  'merit-promotion':'merit-promotions',
                                  'drinking-water-supply-systems':'drinking-water-supplies',
                                  'general-postal-service':'general-postal-services',
                                  'script':'scripts',
                                  'phonetic-alphabetic-writing':'phonetic-alphabetic-writings',
                                  'non-phonetic-alphabetic-writing':'non-phonetic-writings',
                                  'calendar':'calendars',
                                  'scientific-literature':'scientific-literatures',
                                  'history':'histories',
                                  'religious-literature':'religious-literatures',
                                  'fiction':'fictions',
                                  'practical-literature':'practical-literatures',
                                  'philosophy':'philosophies',
                                  'paper-currency':'paper-currencies'})

name_mapping = {}
for col in dataset.raw.columns:
    new_name = col.split("/")[-1]
    name_mapping[col] = new_name.lower()

dataset.raw = dataset.raw.rename(columns=name_mapping)

# find a list of common columns
common_columns = list(set(raw_dat.columns).intersection(set(dataset.raw.columns)))

# find a list of columns that are in the social complexity dataset 
from mappings import social_complexity_mapping
sc_raw_cols = []
for key, dic in social_complexity_mapping.items():
    sc_raw_cols.extend(dic.keys())

common_columns = ["polityname","year"] + (list(set(common_columns).intersection(set(sc_raw_cols))))

# create a dataframe with the common columns and a column to indicate the source of the data
df1 = raw_dat[common_columns]
df1['data'] = 'old'
df2 = dataset.raw[common_columns]
df2['data'] = 'new'
df_merged = pd.concat([df1, df2])


/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_33035/2510997455.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['data'] = 'old'
/var/folders/c3/82t48qlj2bg0xttc4k7vx70r0000gq/T/ipykernel_33035/2510997455.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['data'] = 'new'


In [28]:
# group by polityname
df_merged_pol = df_merged.groupby('polityname')
new_politites = []
# for each polity check column by column if the data is the same between new and old datasets
for polity in df_merged.polityname.unique():
    print(polity)
    df_temp = df_merged_pol.get_group(polity)

    if len(df_temp.loc[df_temp.data == 'old']) != len(df_temp.loc[df_temp.data == 'new']):
        if len(df_temp.loc[df_temp.data == 'old']) == 0:
            print("no old data points for", polity)
            new_politites.append(polity)
            continue
        elif len(df_temp.loc[df_temp.data == 'new']) > len(df_temp.loc[df_temp.data == 'old']):
            # find common years
            years = list(set(df_temp.loc[df_temp.data == 'old'].year).intersection(set(df_temp.loc[df_temp.data == 'new'].year)))
            df_temp = df_temp.loc[df_temp.year.isin(years)]

            compare(df_temp.loc[df_temp.data == 'old'], df_temp.loc[df_temp.data == 'new'], common_columns)
    else:
        compare(df_temp.loc[df_temp.data == 'old'], df_temp.loc[df_temp.data == 'new'], common_columns)


iq_abbasid_cal_1
same values for merit-promotions
same values for population-of-the-largest-settlements
same values for judges
different lengths for precious-metals
old data
0    1.0
1    1.0
Name: precious-metals, dtype: float64
new data
Series([], Name: precious-metals, dtype: float64)



different lengths for nonwritten-records
old data
0    1.0
1    1.0
Name: nonwritten-records, dtype: float64
new data
Series([], Name: nonwritten-records, dtype: float64)



different lengths for administrative-levels
old data
0    6.0
1    6.0
Name: administrative-levels, dtype: float64
new data
Series([], Name: administrative-levels, dtype: float64)



no values for tokens
different lengths for military-levels
old data
0    6.0
1    6.0
Name: military-levels, dtype: float64
new data
Series([], Name: military-levels, dtype: float64)



different lengths for polity-populations
old data
0    23000000.0
1     9000000.0
Name: polity-populations, dtype: float64
new data
1    10000000.0
Name: polity-popu

Differences between the datasets are related to the different ways in which the the dataset imports disputed, uncertain and variables with ranges. Old code samples while the new code takes the mean.